In [101]:
%reset
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import numpy as np
#import matplotlib.pyplot as plt
import torch

import defmod as dm

#torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
dim = 2
nb_pts = 10
sigma = 1
manifold = dm.manifold.Landmarks(dim, nb_pts)

gd = torch.rand(nb_pts, dim)
manifold.fill_gd(gd.view(-1))

globaltrans = dm.deformationmodules.GlobalTranslation(manifold, sigma)

In [103]:
manifold_trans = dm.manifold.Landmarks(dim, 1)
manifold_trans.fill_gd(globaltrans.z().view(-1))
trans = dm.deformationmodules.Translations(manifold_trans, sigma)

In [104]:
manifold.numel_gd

20

In [105]:
points = torch.rand(6,dim)
globaltrans(points)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<MmBackward>)

In [106]:
points, globaltrans.manifold.gd

(tensor([[0.2295, 0.4394],
         [0.2710, 0.2232],
         [0.8762, 0.5600],
         [0.4527, 0.5730],
         [0.3992, 0.9045],
         [0.1558, 0.7996]]),
 tensor([0.2066, 0.6516, 0.1269, 0.0066, 0.6987, 0.1966, 0.5904, 0.8483, 0.2622,
         0.0973, 0.4778, 0.7322, 0.4836, 0.5332, 0.8659, 0.8079, 0.8990, 0.2138,
         0.2904, 0.0749]))

In [107]:
globaltrans.z()

tensor([[0.4902, 0.4162]])

In [108]:
globaltrans.cost()

tensor(0., grad_fn=<MulBackward0>)

In [109]:
globaltrans.fill_controls_zero()

In [121]:
cont = torch.rand(1,dim)
globaltrans.fill_controls(cont)
globaltrans.controls

tensor([[0.5030, 0.8921]])

In [122]:
globaltrans.cost()

tensor(0.5245, grad_fn=<MulBackward0>)

In [123]:
trans.fill_controls(cont)
trans.manifold.gd
trans.cost()

tensor(0.5245)

In [124]:
trans.controls

tensor([[0.5030, 0.8921]])

In [132]:
globaltrans.compute_geodesic_control(manifold)

In [133]:
globaltrans.controls

tensor([0., 0.], grad_fn=<ViewBackward>)

In [134]:
trans.compute_geodesic_control(manifold)
trans.controls

tensor([0., 0.], grad_fn=<ViewBackward>)